## Objective of the Project

In this project, the objective is to predict if a crime is an homicide or not. For that reason 2 databases will be used. The first one is a database of crimes in DC and the second one is a weather temperature database. It's important to check if weather temperature is related to homicides. For that reason, it will be necessary to clean the databases, prepare them for modeling, create the models and finally check results.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Washington Crime Database - 33910 entries 

In [2]:
cr_raw = pd.read_csv('Crime_Incidents_in_2019.csv')
cr_raw.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-76.982944,38.887599,10199597,2019-11-07T11:41:36.000Z,DAY,OTHERS,THEFT/OTHER,1500 - 1599 BLOCK OF INDEPENDENCE AVENUE SE,401480.0,135528.0,...,006801 2,6801.0,Precinct 87,38.887592,-76.982941,NaN,2019-11-07T10:36:52.000Z,2019-11-07T11:42:02.000Z,429611163,10199597-01
1,-77.010378,38.820469,17084415,2019-01-28T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,130 - 199 BLOCK OF IRVINGTON STREET SW,399099.0,128076.0,...,010900 2,10900.0,Precinct 126,38.820461,-77.010375,NaN,2017-05-19T22:58:53.000Z,2017-05-20T02:26:45.000Z,429841378,17084415-01
2,-76.952665,38.920544,18208996,2019-03-22T16:18:15.000Z,EVENING,OTHERS,THEFT/OTHER,2400 BLOCK OF MARKET STREET NE,404105.0,139186.0,...,009000 1,9000.0,Precinct 139,38.920536,-76.952663,NaN,2018-12-09T17:01:49.000Z,2018-12-09T18:49:21.000Z,429890611,18208996-01
3,-77.027565,38.897353,18221681,2019-01-01T10:24:06.000Z,DAY,OTHERS,THEFT/OTHER,1100 - 1199 BLOCK OF F STREET NW,397609.0,136611.0,...,005800 1,5800.0,Precinct 129,38.897346,-77.027563,DOWNTOWN,2018-12-31T11:49:19.000Z,2018-12-31T14:43:21.000Z,429890721,18221681-01
4,-77.021929,38.899129,18221708,2019-01-01T15:48:01.000Z,EVENING,OTHERS,THEFT/OTHER,700 - 799 BLOCK OF 7TH STREET NW,398098.0,136808.0,...,005800 1,5800.0,Precinct 129,38.899121,-77.021926,DOWNTOWN,2018-12-31T12:48:46.000Z,2018-12-31T12:51:47.000Z,429890728,18221708-01


In [3]:
# Variables
cr_raw.columns

Index(['X', 'Y', 'CCN', 'REPORT_DAT', 'SHIFT', 'METHOD', 'OFFENSE', 'BLOCK',
       'XBLOCK', 'YBLOCK', 'WARD', 'ANC', 'DISTRICT', 'PSA',
       'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT',
       'VOTING_PRECINCT', 'LATITUDE', 'LONGITUDE', 'BID', 'START_DATE',
       'END_DATE', 'OBJECTID', 'OCTO_RECORD_ID'],
      dtype='object')

## Choose best columns for the model

In [4]:
cr_raw_01 = cr_raw.drop(['OCTO_RECORD_ID', 'X', 'Y', 'BLOCK', 'XBLOCK', 'YBLOCK', 'VOTING_PRECINCT', 
                         'BLOCK_GROUP', 'CENSUS_TRACT', 'NEIGHBORHOOD_CLUSTER', 'REPORT_DAT', 'CCN',
                        'ANC', 'END_DATE', 'BID', 'DISTRICT'], axis=1)
print(cr_raw_01.columns)

Index(['SHIFT', 'METHOD', 'OFFENSE', 'WARD', 'PSA', 'LATITUDE', 'LONGITUDE',
       'START_DATE', 'OBJECTID'],
      dtype='object')


In [5]:
# OBJECTID as index
cr_raw_02 = cr_raw_01.set_index('OBJECTID')
cr_raw_02.head()

,SHIFT,METHOD,OFFENSE,WARD,PSA,LATITUDE,LONGITUDE,START_DATE
OBJECTID,,,,,,,,
429611163,DAY,OTHERS,THEFT/OTHER,6,107.0,38.887592,-76.982941,2019-11-07T10:36:52.000Z
429841378,MIDNIGHT,GUN,HOMICIDE,8,708.0,38.820461,-77.010375,2017-05-19T22:58:53.000Z
429890611,EVENING,OTHERS,THEFT/OTHER,5,503.0,38.920536,-76.952663,2018-12-09T17:01:49.000Z
429890721,DAY,OTHERS,THEFT/OTHER,2,209.0,38.897346,-77.027563,2018-12-31T11:49:19.000Z
429890728,EVENING,OTHERS,THEFT/OTHER,2,101.0,38.899121,-77.021926,2018-12-31T12:48:46.000Z


In [6]:
# Fix START_DATE Column format
start_date = cr_raw_02['START_DATE']
start_date_split = start_date.iloc[:].str.split('T')
start_date_split = pd.DataFrame(start_date_split)

start_date_split[['Date', 'Time']] = pd.DataFrame(start_date_split.START_DATE.values.tolist(), 
                                                  index=start_date_split.index)
start_date_split_cols = start_date_split.drop('START_DATE', axis=1)
start_date_split_cols.head()

,Date,Time
OBJECTID,,
429611163,2019-11-07,10:36:52.000Z
429841378,2017-05-19,22:58:53.000Z
429890611,2018-12-09,17:01:49.000Z
429890721,2018-12-31,11:49:19.000Z
429890728,2018-12-31,12:48:46.000Z


In [7]:
# Fix Time format
time_split = start_date_split_cols['Time'].str.split('.')
time_split = pd.DataFrame(time_split)

start_date_split_cols[['Time', 'Trash']] = pd.DataFrame(time_split.Time.values.tolist(), 
                                                  index=start_date_split_cols.index)
start_date_split_cols = start_date_split_cols.drop('Trash', axis=1)
start_date_split_cols.head()

,Date,Time
OBJECTID,,
429611163,2019-11-07,10:36:52
429841378,2017-05-19,22:58:53
429890611,2018-12-09,17:01:49
429890721,2018-12-31,11:49:19
429890728,2018-12-31,12:48:46


In [8]:
# Merge changes to dataframe
start_date_split_date = pd.DataFrame(start_date_split_cols['Date'].str.split('-'))
cr_raw_02[['YEAR', 'MONTH', 'DAY']] = pd.DataFrame(start_date_split_date.Date.values.tolist(), 
                                                  index=cr_raw_02.index)
cr_raw_02['TIME'] = start_date_split_cols['Time']

cr_raw_03 = cr_raw_02.drop('START_DATE', axis=1)
cr_raw_03['DATE'] = start_date_split_cols['Date']
cr_raw_03.head()

,SHIFT,METHOD,OFFENSE,WARD,PSA,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,TIME,DATE
OBJECTID,,,,,,,,,,,,
429611163,DAY,OTHERS,THEFT/OTHER,6,107.0,38.887592,-76.982941,2019,11,07,10:36:52,2019-11-07
429841378,MIDNIGHT,GUN,HOMICIDE,8,708.0,38.820461,-77.010375,2017,05,19,22:58:53,2017-05-19
429890611,EVENING,OTHERS,THEFT/OTHER,5,503.0,38.920536,-76.952663,2018,12,09,17:01:49,2018-12-09
429890721,DAY,OTHERS,THEFT/OTHER,2,209.0,38.897346,-77.027563,2018,12,31,11:49:19,2018-12-31
429890728,EVENING,OTHERS,THEFT/OTHER,2,101.0,38.899121,-77.021926,2018,12,31,12:48:46,2018-12-31


## Clean NaN Values

In [9]:
cr_clean = cr_raw_03.dropna()
len(cr_clean)

33908

In [10]:
# Work with entries of 2019
cr_clean = cr_clean[cr_clean['YEAR']=='2019']
cr_clean.head()

,SHIFT,METHOD,OFFENSE,WARD,PSA,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,TIME,DATE
OBJECTID,,,,,,,,,,,,
429611163,DAY,OTHERS,THEFT/OTHER,6,107.0,38.887592,-76.982941,2019,11,07,10:36:52,2019-11-07
429890792,EVENING,OTHERS,THEFT F/AUTO,5,502.0,38.911104,-77.007806,2019,01,03,14:34:06,2019-01-03
429890793,EVENING,OTHERS,MOTOR VEHICLE THEFT,6,101.0,38.901915,-77.015183,2019,01,03,15:23:19,2019-01-03
429890794,EVENING,OTHERS,THEFT/OTHER,5,504.0,38.938309,-76.989285,2019,01,02,20:00:28,2019-01-02
429890795,EVENING,OTHERS,THEFT F/AUTO,5,406.0,38.953704,-76.988013,2019,01,03,15:10:54,2019-01-03


## Washington Weather Database

In [11]:
wea_raw = pd.read_csv('SD_b2dates.csv')
wea_raw.head()

,Date,PRCP,SNOW,SNWD,TMAX,TMIN,MEAN
0,2019-01-01,1.03,0,0,36.0,31.0,33.5
1,2019-01-02,0.01,T,0,32.0,27.0,29.5
2,2019-01-03,T,T,0,40.0,23.0,31.5
3,2019-01-04,0,0,0,42.0,24.0,33.0
4,2019-01-05,0,0,0,53.0,32.0,42.5


In [12]:
# Only the Date and Mean column will be used
wea_clean = wea_raw.drop(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN'], axis=1)
wea_clean = wea_clean.rename({'Date': 'Date', 'MEAN': 'WEATHER_TEMP'}, axis=1)
wea_clean.head()

,Date,WEATHER_TEMP
0,2019-01-01,33.5
1,2019-01-02,29.5
2,2019-01-03,31.5
3,2019-01-04,33.0
4,2019-01-05,42.5


## Merge Crime and Weather Databases

In [13]:
final_db = pd.merge(cr_clean, wea_clean, how='left', left_on='DATE', right_on='Date')
final_db = final_db.drop('Date', axis=1)
final_db.head()

,SHIFT,METHOD,OFFENSE,WARD,PSA,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,TIME,DATE,WEATHER_TEMP
0,DAY,OTHERS,THEFT/OTHER,6,107.0,38.887592,-76.982941,2019,11,07,10:36:52,2019-11-07,31.0
1,EVENING,OTHERS,THEFT F/AUTO,5,502.0,38.911104,-77.007806,2019,01,03,14:34:06,2019-01-03,31.5
2,EVENING,OTHERS,MOTOR VEHICLE THEFT,6,101.0,38.901915,-77.015183,2019,01,03,15:23:19,2019-01-03,31.5
3,EVENING,OTHERS,THEFT/OTHER,5,504.0,38.938309,-76.989285,2019,01,02,20:00:28,2019-01-02,29.5
4,EVENING,OTHERS,THEFT F/AUTO,5,406.0,38.953704,-76.988013,2019,01,03,15:10:54,2019-01-03,31.5


In [14]:
# dummy variable OFFENSE
dummy = final_db['OFFENSE']=='HOMICIDE'
#dummy_REF = dummy_REF*1 # convert TRUE/FALSE and then 1/0
dummy = dummy*1
final_db['TARGET_HOMICIDE'] = dummy
final_db.head()

,SHIFT,METHOD,OFFENSE,WARD,PSA,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,TIME,DATE,WEATHER_TEMP,TARGET_HOMICIDE
0,DAY,OTHERS,THEFT/OTHER,6,107.0,38.887592,-76.982941,2019,11,07,10:36:52,2019-11-07,31.0,0
1,EVENING,OTHERS,THEFT F/AUTO,5,502.0,38.911104,-77.007806,2019,01,03,14:34:06,2019-01-03,31.5,0
2,EVENING,OTHERS,MOTOR VEHICLE THEFT,6,101.0,38.901915,-77.015183,2019,01,03,15:23:19,2019-01-03,31.5,0
3,EVENING,OTHERS,THEFT/OTHER,5,504.0,38.938309,-76.989285,2019,01,02,20:00:28,2019-01-02,29.5,0
4,EVENING,OTHERS,THEFT F/AUTO,5,406.0,38.953704,-76.988013,2019,01,03,15:10:54,2019-01-03,31.5,0


In [15]:
final_db.to_csv('clean_db_p2.csv')

In [16]:
db = pd.read_csv('clean_db_p2.csv')
# Eliminate columns for the model
db = db.drop(['Unnamed: 0', 'LATITUDE', 'LONGITUDE', 'OFFENSE'], axis=1)
db.head()

,SHIFT,METHOD,WARD,PSA,YEAR,MONTH,DAY,TIME,DATE,WEATHER_TEMP,TARGET_HOMICIDE
0,DAY,OTHERS,6,107.0,2019,11,7,10:36:52,2019-11-07,31.0,0
1,EVENING,OTHERS,5,502.0,2019,1,3,14:34:06,2019-01-03,31.5,0
2,EVENING,OTHERS,6,101.0,2019,1,3,15:23:19,2019-01-03,31.5,0
3,EVENING,OTHERS,5,504.0,2019,1,2,20:00:28,2019-01-02,29.5,0
4,EVENING,OTHERS,5,406.0,2019,1,3,15:10:54,2019-01-03,31.5,0


## Function to get the day number from the date. Monday=0, Tuesday=1, ...

In [17]:
def dia_semana(fecha):
    return datetime.strptime(fecha, '%Y-%m-%d').date().weekday()

In [18]:
# Apply function to the dataframe column DAY
db['DAY'] = db['DATE'].apply(dia_semana)
db.head()

,SHIFT,METHOD,WARD,PSA,YEAR,MONTH,DAY,TIME,DATE,WEATHER_TEMP,TARGET_HOMICIDE
0,DAY,OTHERS,6,107.0,2019,11,3,10:36:52,2019-11-07,31.0,0
1,EVENING,OTHERS,5,502.0,2019,1,3,14:34:06,2019-01-03,31.5,0
2,EVENING,OTHERS,6,101.0,2019,1,3,15:23:19,2019-01-03,31.5,0
3,EVENING,OTHERS,5,504.0,2019,1,2,20:00:28,2019-01-02,29.5,0
4,EVENING,OTHERS,5,406.0,2019,1,3,15:10:54,2019-01-03,31.5,0


In [19]:
# Drop unnecessary columns
db_01 = db.drop(['YEAR', 'MONTH', 'DATE'], axis=1)
db_01['PSA'] = db_01['PSA'].astype(int)
db_01.head()

,SHIFT,METHOD,WARD,PSA,DAY,TIME,WEATHER_TEMP,TARGET_HOMICIDE
0,DAY,OTHERS,6,107,3,10:36:52,31.0,0
1,EVENING,OTHERS,5,502,3,14:34:06,31.5,0
2,EVENING,OTHERS,6,101,3,15:23:19,31.5,0
3,EVENING,OTHERS,5,504,2,20:00:28,29.5,0
4,EVENING,OTHERS,5,406,3,15:10:54,31.5,0


## Only get the hour of the Time Variable

In [20]:
def hora(x):
    return int(x[:2])
db_01['TIME'] = db_01['TIME'].apply(hora)
db_01.head()

,SHIFT,METHOD,WARD,PSA,DAY,TIME,WEATHER_TEMP,TARGET_HOMICIDE
0,DAY,OTHERS,6,107,3,10,31.0,0
1,EVENING,OTHERS,5,502,3,14,31.5,0
2,EVENING,OTHERS,6,101,3,15,31.5,0
3,EVENING,OTHERS,5,504,2,20,29.5,0
4,EVENING,OTHERS,5,406,3,15,31.5,0


In [21]:
# Use LabelEncoder to transform non-numeric variables to nummeric
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [22]:
# Transform column SHIFT to values [0 1 2]
db_01.iloc[:, 0] = labelencoder.fit_transform(db_01.iloc[:, 0].values)
print(db_01['SHIFT'].unique());

[0 1 2]


In [23]:
# Transform columna METHOD to values [2 0 1]
db_01.iloc[:, 1] = labelencoder.fit_transform(db_01.iloc[:, 1].values)
print(db_01['METHOD'].unique())

[2 0 1]


## Final Dataframe for the model

In [24]:
db_model = db_01.copy()
db_model.head()

,SHIFT,METHOD,WARD,PSA,DAY,TIME,WEATHER_TEMP,TARGET_HOMICIDE
0,0,2,6,107,3,10,31.0,0
1,1,2,5,502,3,14,31.5,0
2,1,2,6,101,3,15,31.5,0
3,1,2,5,504,2,20,29.5,0
4,1,2,5,406,3,15,31.5,0


## Variables and labels

In [25]:
# Variables predictoras
X = db_model.copy()
X = X.drop(['TARGET_HOMICIDE'], axis=1)
X = X.values

y = db_model['TARGET_HOMICIDE'].values

In [26]:
# Split train and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Models

In [27]:
# Four models will be compared, the ml_models function contains all of them
def ml_models(X_train, y_train):
    
    models = []
    
    # K-Nearest Neightbors
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    models.append(knn)
    
    # Arbol de decision
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(random_state=0)
    tree.fit(X_train, y_train)
    models.append(tree)   
  
    # Random forest
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(random_state=0)
    forest.fit(X_train, y_train)
    models.append(forest)
    
    # Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gauss = GaussianNB()
    gauss.fit(X_train, y_train)
    models.append(gauss)
    
    return models

In [28]:
# Train all models with train set
ml_models(X_train, y_train)

[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                      weights='uniform'),
 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=0, splitter='best'),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                     

## Results with training data

In [29]:
for i in ml_models(X_train, y_train):
    print(f'{type(i).__name__}: {i.score(X_train, y_train)}')

KNeighborsClassifier: 0.9954628333969384
DecisionTreeClassifier: 1.0
RandomForestClassifier: 1.0
GaussianNB: 0.9661196624687275


### Note:
All models work wel, but DecisionTreeClassifier and RandomForestClassifier are the best.

## Accuracy of the models with the test set

In [30]:
from sklearn.metrics import confusion_matrix
for i in ml_models(X_train, y_train):
    cm = confusion_matrix(y_test, i.predict(X_test))
    
    TN, FP, FN, TP = confusion_matrix(y_test, i.predict(X_test)).ravel()
    print(f'Confusion Matrix of model {type(i).__name__}')
    print(cm)
    
    print(f'{type(i).__name__} Accuracy = ', (TP+TN)/(TP + TN + FN + FP))
    print('------------------------------------------------------ \n' )

Confusion Matrix of model KNeighborsClassifier
[[10050     1]
 [   56     0]]
KNeighborsClassifier Accuracy =  0.9943603443158208
------------------------------------------------------ 

Confusion Matrix of model DecisionTreeClassifier
[[10019    32]
 [   35    21]]
DecisionTreeClassifier Accuracy =  0.9933709310378945
------------------------------------------------------ 

Confusion Matrix of model RandomForestClassifier
[[10047     4]
 [   47     9]]
RandomForestClassifier Accuracy =  0.9949539922825764
------------------------------------------------------ 

Confusion Matrix of model GaussianNB
[[9701  350]
 [  11   45]]
GaussianNB Accuracy =  0.9642821806668646
------------------------------------------------------ 



# Note:
# The best model is Random Forest.

## Importance of variables in prediction:

In [31]:
X = db_model.copy()
X = X.drop(['TARGET_HOMICIDE'], axis=1)
importance = pd.DataFrame({'variables': X.columns, 'imp': np.round(ml_models(X_train, y_train)[2].feature_importances_, 4)})
importance = importance.sort_values('imp', ascending=False).set_index('variables')
importance

,imp
variables,
WEATHER_TEMP,0.2658
TIME,0.2447
PSA,0.1822
DAY,0.1023
METHOD,0.0959
SHIFT,0.0598
WARD,0.0492


### Note:
The top  variables in importance of prediction are: Weather Temperature, Time of crime and PSA (location of crime).